In [1]:
from typing import List, Tuple
from datetime import datetime, timedelta
import numpy.typing as npt
import numpy as np
import os
import cv2
import pickle
from tqdm import tqdm
import itertools
import json

from utils.compose_factory import compose_factory
from utils.data_dirs import data_dirs
from utils.get_videos import get_videos

In [2]:
raw_dir, processed_dir, folder, EXPERIMENT_DATA, suffix, *_ = data_dirs(False)
print(raw_dir)
print(processed_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Mini
/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Mini
v1.0-mini
/work/apperception/data/raw/scenic/experiment_data



In [3]:
fps = 20

In [4]:
interpolate = False

In [5]:
partition_dir = os.path.join(processed_dir, 'partition')

In [6]:
compose = compose_factory(fps, raw_dir, processed_dir, interpolate)

In [ ]:
for l in os.listdir(partition_dir):
    d = os.path.join(partition_dir, l)
    with open(os.path.join(d, 'sample_data.pickle'), "rb") as f:
        sample_data = pickle.load(f)
    videos = get_videos(sample_data)
    
    
    output = {}
    for name in videos:
        video = videos[name]
        frames, filename, start = compose(l, name, video)
        output[name] = {
            "filename": filename,
            "start": start,
            "columns": [
                "cameraId",
                "frameId",
                "frameNum",
                "filename",
                "cameraTranslation",
                "cameraRotation",
                "cameraIntrinsic",
                "egoTranslation",
                "egoRotation",
                "timestamp",
                "cameraHeading",
                "egoHeading",
            ],
            "frames": video
        }
        print("done", filename)
    base = os.path.join(processed_dir, 'videos', l)
    with open(os.path.join(base, 'frames.pickle'), "wb") as f:
        pickle.dump(output, f)
    
    with open(os.path.join(base, 'frames.json'), "w") as f:
        json.dump(output, f, indent=2, sort_keys=True, default=str)

Reading scene: scene-1077-CAM_FRONT_LEFT


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [13:32<00:00,  2.03s/it]


Writing scene (/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Mini/videos/singapore-hollandvillage/scene-1077-CAM_FRONT_LEFT-2018-11-21-3-39-27-904844.mp4):


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:05<00:00, 67.46it/s]


done scene-1077-CAM_FRONT_LEFT-2018-11-21-3-39-27-904844.mp4
Reading scene: scene-1077-CAM_FRONT


 34%|████████████████████████████████████████████████████▍                                                                                                       | 134/399 [09:16<24:53,  5.64s/it]